# Anomaly Detection Group Project

1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
7. Which lessons are least accessed?
8. Anything else I should be aware of?

# acquire

In [1]:
import pandas as pd
import os
from env import *

In [2]:
def df_to_csv(df):
    try:
        df.to_csv(filename, index=False, mode='x')
    except FileExistsError:
        df.to_csv(filename,index=False)

In [3]:
def offline_lesson_kernel_restart():
    exists = os.path.isfile(filename)
    if exists:
        df = pd.read_csv(filename)
        
        return df
    else:
        #Define query
        query = '''
                select * 
                from `logs`
                JOIN cohorts ON logs.cohort_id=cohorts.id;
                '''
        #Define url
        url = get_db_url('curriculum_logs')
        
        #Read data from SQL server
        df = pd.read_sql(query, url)
        
        #Cache
        df_to_csv(df)
        
        return df

In [ ]:
(CAST(logs.date as DATETIME) + CAST(logs.time AS TIME))as 'date_time'

In [4]:
filename = "curriculum_logs.csv"

In [5]:
df = offline_lesson_kernel_restart()

In [6]:
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2


In [7]:
df['fixed_date']=df['date']+ ' ' +df['time']

In [8]:
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,fixed_date
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,2018-01-26 09:55:03
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,2018-01-26 09:56:02
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,2018-01-26 09:56:05
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,2018-01-26 09:56:06
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2,2018-01-26 09:56:24


In [9]:
pd.to_datetime(df['fixed_date'])

0        2018-01-26 09:55:03
1        2018-01-26 09:56:02
2        2018-01-26 09:56:05
3        2018-01-26 09:56:06
4        2018-01-26 09:56:24
                 ...        
847325   2021-04-21 16:41:51
847326   2021-04-21 16:42:02
847327   2021-04-21 16:42:09
847328   2021-04-21 16:44:37
847329   2021-04-21 16:44:39
Name: fixed_date, Length: 847330, dtype: datetime64[ns]

In [10]:
df.set_index(pd.DatetimeIndex(df['fixed_date']), inplace=True)

In [ ]:
df.head()

In [11]:
df = df.drop(columns = ['date','time','fixed_date'])

In [12]:
df['data']= df['program_id']== 3
df['web']= df['program_id']== 2
df['php']= df['program_id']== 1
df['front_end']= df['program_id']== 4

In [ ]:
data_df['cohort_id'] = data_df['cohort_id'].astype(int)

In [13]:
df.head()

,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,data,web,php,front_end
fixed_date,,,,,,,,,,,,,,,,,
2018-01-26 09:55:03,/,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,False,False,True,False
2018-01-26 09:56:02,java-ii,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,False,False,True,False
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,False,False,True,False
2018-01-26 09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1,False,False,True,False
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2,False,True,False,False


# 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?

javascript-i

### Most Accessed Path

In [14]:
df['data'].value_counts()

False    743918
True     103412
Name: data, dtype: int64

### Data Science Subset DataFrame

each cohort, sort by count (top 10)

each path is in list?

In [36]:
data_df = df[df['data']==True].copy()

In [19]:
data_df.nunique()

path          682
user_id       111
cohort_id       5
ip            990
id              5
name            5
slack           5
start_date      5
end_date        5
created_at      5
updated_at      5
deleted_at      0
program_id      1
data            1
web             1
php             1
front_end       1
dtype: int64

In [63]:
data_df = data_df[data_df['path']!= '/']

In [64]:
data_df.head()

,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,data,web,php,front_end
fixed_date,,,,,,,,,,,,,,,,,
2019-08-20 09:40:28,3-sql/1-mysql-overview,470,34,97.105.19.58,34,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,NaN,3,True,False,False,False
2019-08-20 09:40:30,2-storytelling/bad-charts,467,34,97.105.19.58,34,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,NaN,3,True,False,False,False
2019-08-20 09:40:30,2-storytelling/misleading1_baseball.jpg,467,34,97.105.19.58,34,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,NaN,3,True,False,False,False
2019-08-20 09:40:30,2-storytelling/misleading1_fox.jpg,467,34,97.105.19.58,34,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,NaN,3,True,False,False,False
2019-08-20 09:40:30,2-storytelling/misleading3_deaths.jpg,467,34,97.105.19.58,34,Bayes,#,2019-08-19,2020-01-30,2019-08-20 14:38:55,2019-08-20 14:38:55,NaN,3,True,False,False,False


In [65]:
top_results = data_df.groupby(['cohort_id', 'path'])['id'].count().reset_index().sort_values(['cohort_id', 'id'], ascending=[True, False]).groupby('cohort_id').nth(0)

In [66]:
top_results

,path,id
cohort_id,,
34,1-fundamentals/modern-data-scientist.jpg,650
55,6-regression/1-overview,595
59,classification/overview,1109
133,classification/scale_features_or_not.svg,463
137,fundamentals/modern-data-scientist.jpg,627


In [23]:
data_df.groupby(['cohort_id','path']).count()

user_id    ip    id  name  \
cohort_id path                                                                  
34.0      /                                            1967  1967  1967  1967   
          1-fundamentals/1.1-intro-to-data-science      640   640   640   640   
          1-fundamentals/1.2-data-science-pipeline      221   221   221   221   
          1-fundamentals/1.3-pipeline-demo              130   130   130   130   
          1-fundamentals/2.1-excel-overview              86    86    86    86   
...                                                     ...   ...   ...   ...   
137.0     subqueries                                      2     2     2     2   
          timeseries/modeling-lesson2                     1     1     1     1   
          timeseries/overview                             5     5     5     5   
          vocabulary                                      2     2     2     2   
          where                                           1     1     1     1   

                                                    slack  start_date  \
cohort_id path                                                          
34.0      /                                          1967        1967   
          1-fundamentals/1.1-intro-to-data-science    640         640   
          1-fundamentals/1.2-data-science-pipeline    221         221   
          1-fundamentals/1.3-pipeline-demo            130         130   
          1-fundamentals/2.1-excel-overview            86          86   
...                                                   ...         ...   
137.0     subqueries                                    2           2   
          timeseries/modeling-lesson2                   1           1   
          timeseries/overview                           5           5   
          vocabulary                                    2           2   
          where                                         1           1   

                                                    end_date  created_at  \
cohort_id path                                                             
34.0      /                                             1967        1967   
          1-fundamentals/1.1-intro-to-data-science       640         640   
          1-fundamentals/1.2-data-science-pipeline       221         221   
          1-fundamentals/1.3-pipeline-demo               130         130   
          1-fundamentals/2.1-excel-overview               86          86   
...                                                      ...         ...   
137.0     subqueries                                       2           2   
          timeseries/modeling-lesson2                      1           1   
          timeseries/overview                              5           5   
          vocabulary                                       2           2   
          where                                            1           1   

                                                    updated_at  deleted_at  \
cohort_id path                                                               
34.0      /                                               1967           0   
          1-fundamentals/1.1-intro-to-data-science         640           0   
          1-fundamentals/1.2-data-science-pipeline         221           0   
          1-fundamentals/1.3-pipeline-demo                 130           0   
          1-fundamentals/2.1-excel-overview                 86           0   
...                                                        ...         ...   
137.0     subqueries                                         2           0   
          timeseries/modeling-lesson2                        1           0   
          timeseries/overview                                5           0   
          vocabulary                                         2           0   
          where                                              1           0   

                                            

In [51]:
df.groupby(['path','cohort_id']).count().reset_index().sort_values('user_id',ascending=False)

,path,cohort_id,user_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id,data,web,php,front_end
31,/,28.0,6340,6340,6340,6340,6340,6340,6340,6340,6340,0,6340,6340,6340,6340,6340
44,/,59.0,2980,2980,2980,2980,2980,2980,2980,2980,2980,0,2980,2980,2980,2980,2980
27,/,24.0,2101,2101,2101,2101,2101,2101,2101,2101,2101,0,2101,2101,2101,2101,2101
39,/,53.0,2038,2038,2038,2038,2038,2038,2038,2038,2038,0,2038,2038,2038,2038,2038
36,/,34.0,1967,1967,1967,1967,1967,1967,1967,1967,1967,0,1967,1967,1967,1967,1967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5850,content/php_ii/control-structures-i/if.html,22.0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1
5849,content/php_ii/control-structures-i/if.html,18.0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1
5848,content/php_ii/control-structures-i/if.html,17.0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1
5844,content/php_ii/control-structures-i/if.html,12.0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1


In [46]:
top_results = df.groupby(['program_id', 'path'])['id'].count().reset_index().sort_values(['program_id', 'id'], ascending=[True, False]).groupby('program_id').nth(0)

In [47]:
top_results

,path,id
program_id,,
1,/,1681
2,/,35814
3,/,8358
4,content/html-css,2


In [ ]:
df = df[df['path'].str.len()>3]

In [ ]:
df.groupby('path').count().sort_values('user_id',ascending=False)

```python
answer_1 = df.groupby(['program_id', 'path'])
['id'].
count().
reset_index().
sort_values(['program_id', 'id'],
ascending=[True, False])
```

In [ ]:
answer_1 = df.groupby(['program_id', 'path','name'])['id'].count().reset_index().sort_values(['program_id', 'id'],ascending=[True, False])